In [1]:
from PW_explorer.run_clingo import run_clingo
from PW_explorer.load_worlds import load_worlds
from PW_explorer.pwe_helper import rel_slicer
from PW_explorer.time_series import PWETimeSeriesModule
from PW_explorer.pwe_nb_helper import ASPRules
from PW_explorer.export import PWEExport
from PW_explorer.visualize import PWEVisualization

import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
from nxpd import draw
from nxpd import nxpdParams
nxpdParams['show'] = 'ipynb'
from copy import deepcopy
from collections import OrderedDict

import csv
import re
from astropy.time import Time
import random

In [2]:
%load_ext PWE_NB_Extension

In [5]:
tableLayoutCustomer = ['c_custkey','c_name','c_address','c_nationkey','c_phone','c_acctbal','c_mktsement',]#'c_comment']
tableLayoutLineitem = ['l_orderkey','l_partkey','l_suppkey','l_linenumber','l_quantity','l_extendedprice',
                       'l_discount','l_tax','l_returnflag','l_linestatus','l_shipdate','l_commitdate',
                      'l_receiptdate','l_shipinstruct','l_shipmode']#,'l_comment']
tableLayoutOrders = ['o_orderkey','o_custkey','o_orderstatus','o_totalprice','o_orderdate','o_orderpriority',
                     'o_clerk','o_shippriority']#,'o_comment']
tableLayoutPart = ['p_partkey','p_name','p_mfgr','p_brand','p_type','p_size','p_container','p_rentalprice']#,'p_comment']
tableLayoutPartsupp = ['ps_partkey','ps_suppkey','ps_availqty','ps_supplycost']#'ps_comment']
tableLayoutSupllier = ['s_suppkey','s_name','s_address','s_nationkey','s_phone','s_acctbal']#,'s_comment']
tableLayoutNation = ['n_nationkey','n_name','n_regionkey']#,'n_comment']
tableLayoutRegion = ['r_regionkey','r_name']#,'r_comment']

schemas = [tableLayoutCustomer,tableLayoutLineitem,tableLayoutOrders,tableLayoutPart,tableLayoutPartsupp,tableLayoutSupllier,tableLayoutNation,tableLayoutRegion]
schemaNames = ['customer','lineitem','orders','part','partsupp','supplier','nation','region']

In [3]:
# tableLayoutCustomer = ['c_custkey','c_name','c_address','c_nationkey','c_phone','c_acctbal','c_mktsement',
#                        'c_comment']
# tableLayoutLineitem = ['l_orderkey','l_partkey','l_suppkey','l_linenumber','l_quantity','l_extendedprice',
#                        'l_discount','l_tax','l_returnflag','l_linestatus','l_shipdate','l_commitdate',
#                       'l_receiptdate','l_shipinstruct','l_shipmode','l_comment']
# tableLayoutOrders = ['o_orderkey','o_custkey','o_orderstatus','o_totalprice','o_orderdate','o_orderpriority',
#                      'o_clerk','o_shippriority','o_comment']
# tableLayoutPart = ['p_partkey','p_name','p_mfgr','p_brand','p_type','p_size','p_container','p_rentalprice',
#                    'p_comment']
# tableLayoutPartsupp = ['ps_partkey','ps_suppkey','ps_availqty','ps_supplycost','ps_comment']
# tableLayoutSupllier = ['s_suppkey','s_name','s_address','s_nationkey','s_phone','s_acctbal','s_comment']

# schemas = [tableLayoutCustomer,tableLayoutLineitem,tableLayoutOrders,tableLayoutPart,tableLayoutPartsupp,tableLayoutSupllier]
# schemaNames = ['customer','lineitem','orders','part','partsupp','supplier']

In [7]:
customer = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/dbgen/testGen/customer.tbl'
lineitem = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/dbgen/testGen/lineitem.tbl'
orders = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/dbgen/testGen/orders.tbl'
part = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/dbgen/testGen/part.tbl'
partsupp = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/dbgen/testGen/partsupp.tbl'
supplier = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/dbgen/testGen/supplier.tbl'
nation = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/dbgen/testGen/nation.tbl'
region = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/dbgen/testGen/region.tbl'

In [8]:
dataTables = [customer,lineitem,orders,part,partsupp,supplier]
dataValues = []

for db in dataTables:
    singleValues = []
    with open(db) as csvfile:
        readCsv = csv.reader(csvfile,delimiter='|')
        for row in readCsv:
            singleValues.append(row[:-1])
    dataValues.append(singleValues)

In [9]:
dataValues[0]

[['1',
  'Customer#000000001',
  'IVhzIApeRb ot,c,E',
  '15',
  '25-989-741-2988',
  '711.56',
  'BUILDING',
  'to the even, regular platelets. regular, ironic epitaphs nag e'],
 ['2',
  'Customer#000000002',
  'XSTf4,NCwDVaWNe6tEgvwfmRchLXak',
  '13',
  '23-768-687-3665',
  '121.65',
  'AUTOMOBILE',
  'l accounts. blithely ironic theodolites integrate boldly: caref'],
 ['3',
  'Customer#000000003',
  'MG9kdTD2WBHm',
  '1',
  '11-719-748-3364',
  '7498.12',
  'AUTOMOBILE',
  ' deposits eat slyly ironic, even instructions. express foxes detect slyly. blithely even accounts abov'],
 ['4',
  'Customer#000000004',
  'XxVSJsLAGtn',
  '4',
  '14-128-190-5944',
  '2866.83',
  'MACHINERY',
  ' requests. final, regular ideas sleep final accou'],
 ['5',
  'Customer#000000005',
  'KvpyuHCplrB84WgAiGV6sYpZq7Tj',
  '3',
  '13-750-942-6364',
  '794.47',
  'HOUSEHOLD',
  'n accounts will have to unwind. foxes cajole accor'],
 ['6',
  'Customer#000000006',
  'sKZz0CsnMD7mp4Xd0YrBvx,LREYKUWAh yVn',
  '

In [6]:
sqlQueries = []
sqldir = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/dbgen/queries/'
for x in range(1,23):
    sqlfile = sqldir+str(x)+'.sql'
    with open(sqlfile,'r') as f:
        sqlQueries.append(f.read())

In [332]:
def makeOriginal(dataValues,tableName,colour='black'):
    clingoStart = []
    for x in dataValues:
        adjustedValues = [str(int(float(y))) if re.search('[a-zA-Z-]',str(y)) == None else '"'+y.replace(' ','')+'"' for y in x[:-1]]
        adjustedValues = [x.replace('#','') for x in adjustedValues]
        adjustedValues = [x.replace(',','') for x in adjustedValues]
        for i,y in enumerate(adjustedValues):
            if re.search('[-]',str(y)) != None:
                if len(y) == 12:
                    adjustedValues[i] = str(int(Time(y[1:-1], format='isot', scale='utc').mjd))
#                 except:
#                     print(len(y[1:-1]))
#                     print(y[1:-1])
#                     pass
        #adjustedDates = [Time(y, format='isot', scale='utc') if re.search('[-]',str(y)) == None and  else y for y in adjustedValues]
        # replace spaces here?
        clingoStart.append(tableName+'('+','.join(adjustedValues)+').')
    return clingoStart

In [333]:
clingoRules  = []
for i,table in enumerate(schemas):
    clingoRules.append('% schema '+schemaNames[i]+'('+','.join(table).upper()+').')

In [334]:
ASPRules('\n'.join(clingoRules))

'% schema customer(C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEMENT).\n% schema lineitem(L_ORDERKEY,L_PARTKEY,L_SUPPKEY,L_LINENUMBER,L_QUANTITY,L_EXTENDEDPRICE,L_DISCOUNT,L_TAX,L_RETURNFLAG,L_LINESTATUS,L_SHIPDATE,L_COMMITDATE,L_RECEIPTDATE,L_SHIPINSTRUCT,L_SHIPMODE).\n% schema orders(O_ORDERKEY,O_CUSTKEY,O_ORDERSTATUS,O_TOTALPRICE,O_ORDERDATE,O_ORDERPRIORITY,O_CLERK,O_SHIPPRIORITY).\n% schema part(P_PARTKEY,P_NAME,P_MFGR,P_BRAND,P_TYPE,P_SIZE,P_CONTAINER,P_RENTALPRICE).\n% schema partsupp(PS_PARTKEY,PS_SUPPKEY,PS_AVAILQTY,PS_SUPPLYCOST).\n% schema supplier(S_SUPPKEY,S_NAME,S_ADDRESS,S_NATIONKEY,S_PHONE,S_ACCTBAL).'

In [335]:
originalSetup = []
for tab in range(len(dataValues)):
    originalSetup.extend(makeOriginal(dataValues[tab],schemaNames[tab]))

In [336]:
clingoRules.extend(originalSetup)

In [337]:
ASPRules('\n'.join(clingoRules))

'% schema customer(C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEMENT).\n% schema lineitem(L_ORDERKEY,L_PARTKEY,L_SUPPKEY,L_LINENUMBER,L_QUANTITY,L_EXTENDEDPRICE,L_DISCOUNT,L_TAX,L_RETURNFLAG,L_LINESTATUS,L_SHIPDATE,L_COMMITDATE,L_RECEIPTDATE,L_SHIPINSTRUCT,L_SHIPMODE).\n% schema orders(O_ORDERKEY,O_CUSTKEY,O_ORDERSTATUS,O_TOTALPRICE,O_ORDERDATE,O_ORDERPRIORITY,O_CLERK,O_SHIPPRIORITY).\n% schema part(P_PARTKEY,P_NAME,P_MFGR,P_BRAND,P_TYPE,P_SIZE,P_CONTAINER,P_RENTALPRICE).\n% schema partsupp(PS_PARTKEY,PS_SUPPKEY,PS_AVAILQTY,PS_SUPPLYCOST).\n% schema supplier(S_SUPPKEY,S_NAME,S_ADDRESS,S_NATIONKEY,S_PHONE,S_ACCTBAL).\ncustomer(1,"Customer000000001","IVhzIApeRbotcE",15,"25-989-741-2988",711,"BUILDING").\ncustomer(2,"Customer000000002","XSTf4NCwDVaWNe6tEgvwfmRchLXak",13,"23-768-687-3665",121,"AUTOMOBILE").\ncustomer(3,"Customer000000003","MG9kdTD2WBHm",1,"11-719-748-3364",7498,"AUTOMOBILE").\ncustomer(4,"Customer000000004","XxVSJsLAGtn",4,"14-128-190-5944",2866,"MACHINERY").\ncustomer(5,"Customer000000005","KvpyuHCplrB84WgAiGV6sYpZq7Tj",3,"13-750-942-6364",794,"HOUSEHOLD").\ncustomer(6,"Customer000000006","sKZz0CsnMD7mp4Xd0YrBvxLREYKUWAhyVn",20,"30-114-968-4951",7638,"AUTOMOBILE").\ncustomer(7,"Customer000000007","TcGe5gaZNgVePxU5kRrvXBfkasDTea",18,"28-190-982-9759",9561,"AUTOMOBILE").\ncustomer(8,"Customer000000008","I0B10bB0AymmC0PrRYBCP1yGJ8xcBPmWhl5",17,"27-147-574-9335",6819,"BUILDING").\ncustomer(9,"Customer000000009","xKiAFTjUsCuxfeleNqefumTrjS",8,"18-338-906-3675",8324,"FURNITURE").\ncustomer(10,"Customer000000010","6LrEaV6KR6PLVcgl2ArLQ3rqzLzcT1v2",5,"15-741-346-9870",2753,"HOUSEHOLD").\ncustomer(11,"Customer000000011","PkWS3HlXqwTuzrKg633BEi",23,"33-464-151-3439","-272.60","BUILDING").\ncustomer(12,"Customer000000012","9PWKuhzT4Zr1Q",13,"23-791-276-1263",3396,"HOUSEHOLD").\ncustomer(13,"Customer000000013","nsXQu0oVjD7PM659uC3SRSp",3,"13-761-547-5974",3857,"BUILDING").\ncustomer(14,"Customer000000014","KXkletMlL2JQEA",1,"11-845-129-3851",5266,"FURNITURE").\ncustomer(15,"Customer000000015","YtWggXoOLdwdo7b0yBZaGUQMLJMX1YEC6Dn",23,"33-687-542-7601",2788,"HOUSEHOLD").\ncustomer(16,"Customer000000016","cYiaeMLZSMAOQ2d0W",10,"20-781-609-3107",4681,"FURNITURE").\ncustomer(17,"Customer000000017","izrh6jdqtp2eqdtbkswDD8SG4SzXruMfIXyR7",2,"12-970-682-3487",6,"AUTOMOBILE").\ncustomer(18,"Customer000000018","3txGOAiuFux3zT0Z9NYaFRnZt",6,"16-155-215-1315",5494,"BUILDING").\ncustomer(19,"Customer000000019","uc3bHIx84HwdrmLOjVsiqXCq2tr",18,"28-396-526-5053",8914,"HOUSEHOLD").\ncustomer(20,"Customer000000020","JrPk8Pqplj4Ne",22,"32-957-234-8742",7603,"FURNITURE").\ncustomer(21,"Customer000000021","XYmVpr9yAHDEn",8,"18-902-614-8344",1428,"MACHINERY").\ncustomer(22,"Customer000000022","QI6p41FNs5k7RZoCCVPUTkUdYpB",3,"13-806-545-9701",591,"MACHINERY").\ncustomer(23,"Customer000000023","OdYW13N7Be3OC5MpgfmcYss0Wn6TKT",3,"13-312-472-8245",3332,"HOUSEHOLD").\ncustomer(24,"Customer000000024","HXAFgIAyjxtdqwimt13Y3OZO4xeLe7U8PqG",13,"23-127-851-8031",9255,"MACHINERY").\ncustomer(25,"Customer000000025","Hp8GyFQgGHFYSilH5tBfe",12,"22-603-468-3533",7133,"FURNITURE").\ncustomer(26,"Customer000000026","8ljrc5ZeMl7UciP",22,"32-363-455-4837",5182,"AUTOMOBILE").\ncustomer(27,"Customer000000027","IS8GIyxpBrLpMT0u7",3,"13-137-193-2709",5679,"BUILDING").\ncustomer(28,"Customer000000028","iVyg0daQTha8x2WPWA9m2529m",8,"18-774-241-1462",1007,"FURNITURE").\ncustomer(29,"Customer000000029","sJ5adtfyAkCK63df2vF25zyQMVYE34uh",0,"10-773-203-7342",7618,"FURNITURE").\ncustomer(30,"Customer000000030","nJDsELGAavU63Jl0c5NKsKfL8rIJQQkQnYL2QJY",1,"11-764-165-5076",9321,"BUILDING").\ncustomer(31,"Customer000000031","LUACbO0viaAv6eXOAebryDBxjVst",23,"33-197-837-7094",5236,"HOUSEHOLD").\ncustomer(32,"Customer000000032","jD2xZziUmIdDCtNBLXKj9q0Tlp2iQ6ZcO3J",15,"25-430-914-2194",3471,"BUILDING").\ncustomer(33,"Customer000000033","qFSlMuLucBmx9xnn5ib2csWUwegD",17,"27-375-391-1280","-78.56","AUTOMOBILE").\ncustomer(34,"Customer000000034","Q6G9wZ6dnczmtOx50

In [288]:
endDate = Time('1998-12-01',format='isot', scale='utc')
endDateMjD = endDate.mjd
midDate = int(endDateMjD - random.randint(60,120))

In [289]:
midDate

51030

In [ ]:
tableLayoutLineitem = ['l_orderkey','l_partkey','l_suppkey','l_linenumber','l_quantity','l_extendedprice',
                       'l_discount','l_tax','l_returnflag','l_linestatus','l_shipdate','l_commitdate',
                      'l_receiptdate','l_shipinstruct','l_shipmode']#,'l_comment']

In [ ]:
tableLayoutOrders = ['o_orderkey','o_custkey','o_orderstatus','o_totalprice','o_orderdate','o_orderpriority',
                     'o_clerk','o_shippriority']#,'o_comment']

In [290]:
# 1.sql
sqlCode = 'sqlone(Q,EP,D,RF,LS,SD,T) :- lineitem(_,_,_,_,Q,EP,D,T,RF,LS,SD,_,_,_,_),SD > '+str(midDate)+'.' # needs groupby/orderby


In [17]:
tableLayoutCustomer = ['c_custkey','c_name','c_address','c_nationkey','c_phone','c_acctbal','c_mktsement',]#'c_comment']
tableLayoutLineitem = ['l_orderkey','l_partkey','l_suppkey','l_linenumber','l_quantity','l_extendedprice',
                       'l_discount','l_tax','l_returnflag','l_linestatus','l_shipdate','l_commitdate',
                      'l_receiptdate','l_shipinstruct','l_shipmode']#,'l_comment']
tableLayoutOrders = ['o_orderkey','o_custkey','o_orderstatus','o_totalprice','o_orderdate','o_orderpriority',
                     'o_clerk','o_shippriority']#,'o_comment']
tableLayoutPart = ['p_partkey','p_name','p_mfgr','p_brand','p_type','p_size','p_container','p_rentalprice']#,'p_comment']
tableLayoutPartsupp = ['ps_partkey','ps_suppkey','ps_availqty','ps_supplycost']#'ps_comment']
tableLayoutSupllier = ['s_suppkey','s_name','s_address','s_nationkey','s_phone','s_acctbal']#,'s_comment']
tableLayoutNation = ['n_nationkey','n_name','n_regionkey']#,'n_comment']
tableLayoutRegion = ['r_regionkey','r_name']#,'r_comment']
tableLayoutIntermediate = ['s_suppkey','s_name','s_address','s_nationkey','s_phone','s_acctbal',
                          'p_partkey','p_mfgr','p_type','p_size',
                          'ps_partkey','ps_suppkey','ps_supplycost',
                          'n_name','n_nationkey',
                          'r_regionkey','r_name',]
tableLayoutPenultimate = ['s_suppkey','s_name','s_address','s_nationkey','s_phone','s_acctbal',
                          'p_partkey','p_mfgr','p_type','p_size',
                          'ps_partkey','ps_suppkey','ps_supplycost',
                          'n_name','n_nationkey',
                          'r_regionkey','r_name',]
tableLayoutResult = ['s_acctbal','s_name','n_name','p_partkey','p_mfgr','s_address','s_phone']

schemas = [tableLayoutCustomer,tableLayoutLineitem,tableLayoutOrders,tableLayoutPart,tableLayoutPartsupp,
           tableLayoutSupllier,tableLayoutNation,tableLayoutRegion,tableLayoutIntermediate,tableLayoutPenultimate,
          tableLayoutResult]
schemaNames = ['customer','lineitem','orders','part','partsupp','supplier','nation','region','sqltwo',
               'sqlTwofinal','sqltwoResult']

In [ ]:
# 2.sql so far no nation
sqlCode2Base = 'sqltwo(SSK,SN,SA,SNK,SP,SAB,PPK,PMFGR,PT,PS,PSPK,PSSK,PSSC,NN,NRK,RN,RRK) :- part(PPK,_,PMFGR,_,PT,PS,_,_),supplier(SSK,SN,SA,SNK,SP,SAB),partsupp(PSPK,PSSK,_,PSSC),nation(_,NN,NRK),region(RRK,RN).'

In [11]:
# 2.sql so far no nation
sqlCode2Base = 'sqltwo(SSK,SN,SA,SNK,SP,SAB,PPK,PMFGR,PT,PS,PSPK,PSSK,PSSC,NN,NRK,RN,RRK) :- part(PPK,_,PMFGR,_,PT,PS,_,_),supplier(SSK,SN,SA,SNK,SP,SAB),partsupp(PSPK,PSSK,_,PSSC),nation(_,NN,NRK),region(RRK,RN).'

In [13]:
sqlCode2BaseSecond = 'sqltwo(SSK,SN,SA,SNK,SP,SAB,PPK,PMFGR,PT,PS,PSPK,PSSK,PSSC,NN,NRK,RN,RRK) :- sqltwo(SSK,SN,SA,SNK,SP,SAB,PPK,PMFGR,PT,PS,PSPK,PSSK,PSSC,NN,NRK,RN,RRK)'

In [10]:
sqlCode2Conditions = ['PPK = PSPK', 'SSK = PSSK','PS = 42','PT=STEEL','SNK=NNK','NRK=RRK','RN="ASIA"']#,'PSSC=']

In [16]:
sqlInitial = [sqlCode2Base+','+ x for x in sqlCode2Conditions]
sqlSecondaries = [sqlCode2BaseSecond+','+ x +'.' for x in sqlCode2Conditions]
sqlpenultimate = 'sqlTwofinal(SSK,SN,SA,SNK,SP,SAB,PPK,PMFGR,PT,PS,PSPK,PSSK,PSSC,NN,NRK,RN,RRK) :- sqltwo(SSK,SN,SA,SNK,SP,SAB,PPK,PMFGR,PT,PS,PSPK,PSSK,PSSC,NN,NRK,RN,RRK), '+ ','.join(sqlCode2Conditions)+'.'
finalOperation = 'sqltwoResult(SAB,SN,NN,PPK,PMFGR,SA,SP) :- sqlTwofinal(_,SN,SA,_,SP,SAB,PPK,PMFGR,_,_,_,_,_,NN,_,_,_).'

In [ ]:
#sqlCode3 = 'sqltwo(LOK,LEP,LD,OOD,OSP) :- lineitem(),customer(),orders(),'

In [ ]:
#schema for new values

In [324]:
clingoRules.append(sqlCode)

In [325]:
ASPRules('\n'.join(clingoRules))

'% schema customer(C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEMENT,COLOUR,TRANS,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEMENT).\n% schema lineitem(L_ORDERKEY,L_PARTKEY,L_SUPPKEY,L_LINENUMBER,L_QUANTITY,L_EXTENDEDPRICE,L_DISCOUNT,L_TAX,L_RETURNFLAG,L_LINESTATUS,L_SHIPDATE,L_COMMITDATE,L_RECEIPTDATE,L_SHIPINSTRUCT,L_SHIPMODE,COLOUR,TRANS,L_ORDERKEY,L_PARTKEY,L_SUPPKEY,L_LINENUMBER,L_QUANTITY,L_EXTENDEDPRICE,L_DISCOUNT,L_TAX,L_RETURNFLAG,L_LINESTATUS,L_SHIPDATE,L_COMMITDATE,L_RECEIPTDATE,L_SHIPINSTRUCT,L_SHIPMODE).\n% schema orders(O_ORDERKEY,O_CUSTKEY,O_ORDERSTATUS,O_TOTALPRICE,O_ORDERDATE,O_ORDERPRIORITY,O_CLERK,O_SHIPPRIORITY,COLOUR,TRANS,O_ORDERKEY,O_CUSTKEY,O_ORDERSTATUS,O_TOTALPRICE,O_ORDERDATE,O_ORDERPRIORITY,O_CLERK,O_SHIPPRIORITY).\n% schema part(P_PARTKEY,P_NAME,P_MFGR,P_BRAND,P_TYPE,P_SIZE,P_CONTAINER,P_RENTALPRICE,COLOUR,TRANS,P_PARTKEY,P_NAME,P_MFGR,P_BRAND,P_TYPE,P_SIZE,P_CONTAINER,P_RENTALPRICE).\n% schema partsupp(PS_PARTKEY,PS_SUPPKEY,PS_AVAILQTY,PS_SUPPLYCOST,COLOUR,TRANS,PS_PARTKEY,PS_SUPPKEY,PS_AVAILQTY,PS_SUPPLYCOST).\n% schema supplier(S_SUPPKEY,S_NAME,S_ADDRESS,S_NATIONKEY,S_PHONE,S_ACCTBAL,COLOUR,TRANS,S_SUPPKEY,S_NAME,S_ADDRESS,S_NATIONKEY,S_PHONE,S_ACCTBAL).\ncustomer(1,"Customer000000001","IVhzIApeRbotcE",15,"25-989-741-2988",711,"BUILDING",black,"-",1,"Customer000000001","IVhzIApeRbotcE",15,"25-989-741-2988",711,"BUILDING").\ncustomer(2,"Customer000000002","XSTf4NCwDVaWNe6tEgvwfmRchLXak",13,"23-768-687-3665",121,"AUTOMOBILE",black,"-",2,"Customer000000002","XSTf4NCwDVaWNe6tEgvwfmRchLXak",13,"23-768-687-3665",121,"AUTOMOBILE").\ncustomer(3,"Customer000000003","MG9kdTD2WBHm",1,"11-719-748-3364",7498,"AUTOMOBILE",black,"-",3,"Customer000000003","MG9kdTD2WBHm",1,"11-719-748-3364",7498,"AUTOMOBILE").\ncustomer(4,"Customer000000004","XxVSJsLAGtn",4,"14-128-190-5944",2866,"MACHINERY",black,"-",4,"Customer000000004","XxVSJsLAGtn",4,"14-128-190-5944",2866,"MACHINERY").\ncustomer(5,"Customer000000005","KvpyuHCplrB84WgAiGV6sYpZq7Tj",3,"13-750-942-6364",794,"HOUSEHOLD",black,"-",5,"Customer000000005","KvpyuHCplrB84WgAiGV6sYpZq7Tj",3,"13-750-942-6364",794,"HOUSEHOLD").\ncustomer(6,"Customer000000006","sKZz0CsnMD7mp4Xd0YrBvxLREYKUWAhyVn",20,"30-114-968-4951",7638,"AUTOMOBILE",black,"-",6,"Customer000000006","sKZz0CsnMD7mp4Xd0YrBvxLREYKUWAhyVn",20,"30-114-968-4951",7638,"AUTOMOBILE").\ncustomer(7,"Customer000000007","TcGe5gaZNgVePxU5kRrvXBfkasDTea",18,"28-190-982-9759",9561,"AUTOMOBILE",black,"-",7,"Customer000000007","TcGe5gaZNgVePxU5kRrvXBfkasDTea",18,"28-190-982-9759",9561,"AUTOMOBILE").\ncustomer(8,"Customer000000008","I0B10bB0AymmC0PrRYBCP1yGJ8xcBPmWhl5",17,"27-147-574-9335",6819,"BUILDING",black,"-",8,"Customer000000008","I0B10bB0AymmC0PrRYBCP1yGJ8xcBPmWhl5",17,"27-147-574-9335",6819,"BUILDING").\ncustomer(9,"Customer000000009","xKiAFTjUsCuxfeleNqefumTrjS",8,"18-338-906-3675",8324,"FURNITURE",black,"-",9,"Customer000000009","xKiAFTjUsCuxfeleNqefumTrjS",8,"18-338-906-3675",8324,"FURNITURE").\ncustomer(10,"Customer000000010","6LrEaV6KR6PLVcgl2ArLQ3rqzLzcT1v2",5,"15-741-346-9870",2753,"HOUSEHOLD",black,"-",10,"Customer000000010","6LrEaV6KR6PLVcgl2ArLQ3rqzLzcT1v2",5,"15-741-346-9870",2753,"HOUSEHOLD").\ncustomer(11,"Customer000000011","PkWS3HlXqwTuzrKg633BEi",23,"33-464-151-3439","-272.60","BUILDING",black,"-",11,"Customer000000011","PkWS3HlXqwTuzrKg633BEi",23,"33-464-151-3439","-272.60","BUILDING").\ncustomer(12,"Customer000000012","9PWKuhzT4Zr1Q",13,"23-791-276-1263",3396,"HOUSEHOLD",black,"-",12,"Customer000000012","9PWKuhzT4Zr1Q",13,"23-791-276-1263",3396,"HOUSEHOLD").\ncustomer(13,"Customer000000013","nsXQu0oVjD7PM659uC3SRSp",3,"13-761-547-5974",3857,"BUILDING",black,"-",13,"Customer000000013","nsXQu0oVjD7PM659uC3SRSp",3,"13-761-547-5974",3857,"BUILDING").\ncustomer(14,"Customer000000014","KXkletMlL2JQEA",1,"11-845-129-3851",5266,"FURNITURE",black,"-",14,"Customer000000014","KXkletMlL2JQEA",1,"11-845-129-3851",5266,"FURNITURE").\ncustomer(15,"Customer000000015","YtWg

In [326]:
%clingo -l clingoRules --saveto clingo_soln --save_meta_data_to meta_data --donot-display_input

Output:


'Answer: 1\ncustomer(1,"Customer000000001","IVhzIApeRbotcE",15,"25-989-741-2988",711,"BUILDING",black,"-",1,"Customer000000001","IVhzIApeRbotcE",15,"25-989-741-2988",711,"BUILDING") customer(2,"Customer000000002","XSTf4NCwDVaWNe6tEgvwfmRchLXak",13,"23-768-687-3665",121,"AUTOMOBILE",black,"-",2,"Customer000000002","XSTf4NCwDVaWNe6tEgvwfmRchLXak",13,"23-768-687-3665",121,"AUTOMOBILE") customer(3,"Customer000000003","MG9kdTD2WBHm",1,"11-719-748-3364",7498,"AUTOMOBILE",black,"-",3,"Customer000000003","MG9kdTD2WBHm",1,"11-719-748-3364",7498,"AUTOMOBILE") customer(4,"Customer000000004","XxVSJsLAGtn",4,"14-128-190-5944",2866,"MACHINERY",black,"-",4,"Customer000000004","XxVSJsLAGtn",4,"14-128-190-5944",2866,"MACHINERY") customer(5,"Customer000000005","KvpyuHCplrB84WgAiGV6sYpZq7Tj",3,"13-750-942-6364",794,"HOUSEHOLD",black,"-",5,"Customer000000005","KvpyuHCplrB84WgAiGV6sYpZq7Tj",3,"13-750-942-6364",794,"HOUSEHOLD") customer(6,"Customer000000006","sKZz0CsnMD7mp4Xd0YrBvxLREYKUWAhyVn",20,"30-114-968-4951",7638,"AUTOMOBILE",black,"-",6,"Customer000000006","sKZz0CsnMD7mp4Xd0YrBvxLREYKUWAhyVn",20,"30-114-968-4951",7638,"AUTOMOBILE") customer(7,"Customer000000007","TcGe5gaZNgVePxU5kRrvXBfkasDTea",18,"28-190-982-9759",9561,"AUTOMOBILE",black,"-",7,"Customer000000007","TcGe5gaZNgVePxU5kRrvXBfkasDTea",18,"28-190-982-9759",9561,"AUTOMOBILE") customer(8,"Customer000000008","I0B10bB0AymmC0PrRYBCP1yGJ8xcBPmWhl5",17,"27-147-574-9335",6819,"BUILDING",black,"-",8,"Customer000000008","I0B10bB0AymmC0PrRYBCP1yGJ8xcBPmWhl5",17,"27-147-574-9335",6819,"BUILDING") customer(9,"Customer000000009","xKiAFTjUsCuxfeleNqefumTrjS",8,"18-338-906-3675",8324,"FURNITURE",black,"-",9,"Customer000000009","xKiAFTjUsCuxfeleNqefumTrjS",8,"18-338-906-3675",8324,"FURNITURE") customer(10,"Customer000000010","6LrEaV6KR6PLVcgl2ArLQ3rqzLzcT1v2",5,"15-741-346-9870",2753,"HOUSEHOLD",black,"-",10,"Customer000000010","6LrEaV6KR6PLVcgl2ArLQ3rqzLzcT1v2",5,"15-741-346-9870",2753,"HOUSEHOLD") customer(11,"Customer000000011","PkWS3HlXqwTuzrKg633BEi",23,"33-464-151-3439","-272.60","BUILDING",black,"-",11,"Customer000000011","PkWS3HlXqwTuzrKg633BEi",23,"33-464-151-3439","-272.60","BUILDING") customer(12,"Customer000000012","9PWKuhzT4Zr1Q",13,"23-791-276-1263",3396,"HOUSEHOLD",black,"-",12,"Customer000000012","9PWKuhzT4Zr1Q",13,"23-791-276-1263",3396,"HOUSEHOLD") customer(13,"Customer000000013","nsXQu0oVjD7PM659uC3SRSp",3,"13-761-547-5974",3857,"BUILDING",black,"-",13,"Customer000000013","nsXQu0oVjD7PM659uC3SRSp",3,"13-761-547-5974",3857,"BUILDING") customer(14,"Customer000000014","KXkletMlL2JQEA",1,"11-845-129-3851",5266,"FURNITURE",black,"-",14,"Customer000000014","KXkletMlL2JQEA",1,"11-845-129-3851",5266,"FURNITURE") customer(15,"Customer000000015","YtWggXoOLdwdo7b0yBZaGUQMLJMX1YEC6Dn",23,"33-687-542-7601",2788,"HOUSEHOLD",black,"-",15,"Customer000000015","YtWggXoOLdwdo7b0yBZaGUQMLJMX1YEC6Dn",23,"33-687-542-7601",2788,"HOUSEHOLD") customer(16,"Customer000000016","cYiaeMLZSMAOQ2d0W",10,"20-781-609-3107",4681,"FURNITURE",black,"-",16,"Customer000000016","cYiaeMLZSMAOQ2d0W",10,"20-781-609-3107",4681,"FURNITURE") customer(17,"Customer000000017","izrh6jdqtp2eqdtbkswDD8SG4SzXruMfIXyR7",2,"12-970-682-3487",6,"AUTOMOBILE",black,"-",17,"Customer000000017","izrh6jdqtp2eqdtbkswDD8SG4SzXruMfIXyR7",2,"12-970-682-3487",6,"AUTOMOBILE") customer(18,"Customer000000018","3txGOAiuFux3zT0Z9NYaFRnZt",6,"16-155-215-1315",5494,"BUILDING",black,"-",18,"Customer000000018","3txGOAiuFux3zT0Z9NYaFRnZt",6,"16-155-215-1315",5494,"BUILDING") customer(19,"Customer000000019","uc3bHIx84HwdrmLOjVsiqXCq2tr",18,"28-396-526-5053",8914,"HOUSEHOLD",black,"-",19,"Customer000000019","uc3bHIx84HwdrmLOjVsiqXCq2tr",18,"28-396-526-5053",8914,"HOUSEHOLD") customer(20,"Customer000000020","JrPk8Pqplj4Ne",22,"32-957-234-8742",7603,"FURNITURE",black,"-",20,"Customer000000020","JrPk8Pqplj4Ne",22,"32-957-234-8742",7603,"FURNITURE") customer(21,"Customer000000021","XYmVpr9yAHDEn",8,"18-902-614-8344",1428,"MACHINERY",black,"-",21,"Customer00000

In [327]:
pw_rels_dfs, rel_schemas, pw_objs = load_worlds(asp_output=clingo_soln, meta_data=meta_data, reasoner='clingo')

Number of Models: 1


In [328]:
pw_rels_dfs.keys()

dict_keys(['customer_16', 'lineitem_32', 'orders_18', 'part_18', 'partsupp_10', 'supplier_14'])

In [329]:
pw_rels_dfs['customer_16']

,pw,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEMENT,COLOUR,TRANS,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEMENT
0,1,1,"""Customer000000001""","""IVhzIApeRbotcE""",15,"""25-989-741-2988""",711,"""BUILDING""",black,"""-""",1,"""Customer000000001""","""IVhzIApeRbotcE""",15,"""25-989-741-2988""",711,"""BUILDING"""
1,1,2,"""Customer000000002""","""XSTf4NCwDVaWNe6tEgvwfmRchLXak""",13,"""23-768-687-3665""",121,"""AUTOMOBILE""",black,"""-""",2,"""Customer000000002""","""XSTf4NCwDVaWNe6tEgvwfmRchLXak""",13,"""23-768-687-3665""",121,"""AUTOMOBILE"""
2,1,3,"""Customer000000003""","""MG9kdTD2WBHm""",1,"""11-719-748-3364""",7498,"""AUTOMOBILE""",black,"""-""",3,"""Customer000000003""","""MG9kdTD2WBHm""",1,"""11-719-748-3364""",7498,"""AUTOMOBILE"""
3,1,4,"""Customer000000004""","""XxVSJsLAGtn""",4,"""14-128-190-5944""",2866,"""MACHINERY""",black,"""-""",4,"""Customer000000004""","""XxVSJsLAGtn""",4,"""14-128-190-5944""",2866,"""MACHINERY"""
4,1,5,"""Customer000000005""","""KvpyuHCplrB84WgAiGV6sYpZq7Tj""",3,"""13-750-942-6364""",794,"""HOUSEHOLD""",black,"""-""",5,"""Customer000000005""","""KvpyuHCplrB84WgAiGV6sYpZq7Tj""",3,"""13-750-942-6364""",794,"""HOUSEHOLD"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,96,"""Customer000000096""","""vWLOrmXhRR""",8,"""18-422-845-1202""",6323,"""AUTOMOBILE""",black,"""-""",96,"""Customer000000096""","""vWLOrmXhRR""",8,"""18-422-845-1202""",6323,"""AUTOMOBILE"""
96,1,97,"""Customer000000097""","""OApyejbhJG0Iw3jrd1M""",17,"""27-588-919-5638""",2164,"""AUTOMOBILE""",black,"""-""",97,"""Customer000000097""","""OApyejbhJG0Iw3jrd1M""",17,"""27-588-919-5638""",2164,"""AUTOMOBILE"""
97,1,98,"""Customer000000098""","""7yiheXNSpuEAwbswDW""",12,"""22-885-845-6889""","""-551.37""","""BUILDING""",black,"""-""",98,"""Customer000000098""","""7yiheXNSpuEAwbswDW""",12,"""22-885-845-6889""","""-551.37""","""BUILDING"""
98,1,99,"""Customer000000099""","""szsrOiPtCHVS97Lt""",15,"""25-515-237-9232""",4088,"""HOUSEHOLD""",black,"""-""",99,"""Customer000000099""","""szsrOiPtCHVS97Lt""",15,"""25-515-237-9232""",4088,"""HOUSEHOLD"""


In [331]:
pw_rels_dfs['supplier_14']

,pw,S_SUPPKEY,S_NAME,S_ADDRESS,S_NATIONKEY,S_PHONE,S_ACCTBAL,COLOUR,TRANS,S_SUPPKEY,S_NAME,S_ADDRESS,S_NATIONKEY,S_PHONE,S_ACCTBAL
0,1,1,"""Supplier000000001""","""NkD4on9OMIpw3gf0JBoQDd7tgrzrddZ""",17,"""27-918-335-1736""",5755,black,"""-""",1,"""Supplier000000001""","""NkD4on9OMIpw3gf0JBoQDd7tgrzrddZ""",17,"""27-918-335-1736""",5755
1,1,2,"""Supplier000000002""","""89eJ5ksX3ImxJQBvxObC""",5,"""15-679-861-2259""",4032,black,"""-""",2,"""Supplier000000002""","""89eJ5ksX3ImxJQBvxObC""",5,"""15-679-861-2259""",4032
2,1,3,"""Supplier000000003""","""q1G3Pj6OjIuUYfUoH18BFTKP5aU9bEV3""",1,"""11-383-516-1199""",4192,black,"""-""",3,"""Supplier000000003""","""q1G3Pj6OjIuUYfUoH18BFTKP5aU9bEV3""",1,"""11-383-516-1199""",4192
3,1,4,"""Supplier000000004""","""Bk7ah4CK8SYQTepEmvMkkgMwg""",15,"""25-843-787-7479""",4641,black,"""-""",4,"""Supplier000000004""","""Bk7ah4CK8SYQTepEmvMkkgMwg""",15,"""25-843-787-7479""",4641
4,1,5,"""Supplier000000005""","""Gcdm2rJRzl5qlTVzc""",11,"""21-151-690-3663""","""-283.84""",black,"""-""",5,"""Supplier000000005""","""Gcdm2rJRzl5qlTVzc""",11,"""21-151-690-3663""","""-283.84"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,96,"""Supplier000000096""","""Lbxp3WIipyeo2wZme1i9iJxxTt1Mp""",12,"""22-888-654-7193""",3437,black,"""-""",96,"""Supplier000000096""","""Lbxp3WIipyeo2wZme1i9iJxxTt1Mp""",12,"""22-888-654-7193""",3437
96,1,97,"""Supplier000000097""","""MrCQha2G6ndX1fp6CA""",4,"""14-618-678-1789""",6642,black,"""-""",97,"""Supplier000000097""","""MrCQha2G6ndX1fp6CA""",4,"""14-618-678-1789""",6642
97,1,98,"""Supplier000000098""","""ogHn8dpXB5Q""",21,"""31-914-775-1978""",5873,black,"""-""",98,"""Supplier000000098""","""ogHn8dpXB5Q""",21,"""31-914-775-1978""",5873
98,1,99,"""Supplier000000099""","""4SxkTHG28nZrtT0MnVF9H""",18,"""28-272-909-1617""",3671,black,"""-""",99,"""Supplier000000099""","""4SxkTHG28nZrtT0MnVF9H""",18,"""28-272-909-1617""",3671


In [228]:
clingoRulesShort = [clingoRules[0],clingoRules[9]]

In [205]:
clingoRulesShort = ['% schema customer(c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsement,colour,trans,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsement).',
 'customer(4,"Customer000000004","XxVSJsLAGtn",4,"14-128-190-5944",2866,"MACHINERY",black,"-",4,"Customer000000004","XxVSJsLAGtn",4,"14-128-190-5944",2866,"MACHINERY").']

In [157]:
clingoShortNew = []
for x in clingoRulesShort:
    clingoShortNew.append(x.replace(' ',''))

In [171]:
clingoShortNew = clingoRulesShort

In [202]:
print(len(clingoRulesShort[0]))
print(len(clingoRulesShort[1]))

170
172


In [234]:
print(len(clingoRules))

603


In [267]:
print(clingoRules[106],clingoRules[105])

lineitem(1,155190,7706,1,17,21168,0,0,"N","O",50155.0,50125.0,50164.0,"DELIVERINPERSON","TRUCK",black,"-",1,155190,7706,1,17,21168,0,0,"N","O",50155.0,50125.0,50164.0,"DELIVERINPERSON","TRUCK"). customer(100,"Customer#000000100","fptUABXcmkC5Wx",20,"30-749-445-4907",9889,"FURNITURE",black,"-",100,"Customer#000000100","fptUABXcmkC5Wx",20,"30-749-445-4907",9889,"FURNITURE").


In [264]:
clingoRulesShort = clingoRules[0:107]

In [265]:
%clingo -l clingoRulesShort --saveto clingo_soln --save_meta_data_to meta_data --donot-display_input

Output:


'clingo version 5.4.0\nReading from ...hcysbfhcbygweincbsydgibwyebcsygdyc.lp4\nUNKNOWN\n\nModels       : 0+\nCalls        : 1\nTime         : 0.003s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)\nCPU Time     : 0.004s'

In [160]:
pw_rels_dfs, rel_schemas, pw_objs = load_worlds(asp_output=clingo_soln, meta_data=meta_data, reasoner='clingo')

Number of Models: 1


In [161]:
pw_rels_dfs.keys()

dict_keys([])

In [148]:
clingoRulesShort

['% schema customer(c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsement,c_comment).',
 '% schema lineitem(l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment).',
 '% schema orders(o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment).',
 '% schema part(p_partkey,p_name,p_mfgr,p_brand,p_type,p_size,p_container,p_rentalprice,p_comment).',
 '% schema partsupp(ps_partkey,ps_suppkey,ps_availqty,ps_supplycost,ps_comment).',
 '% schema supplier(s_suppkey,s_name,s_address,s_nationkey,s_phone,s_acctbal,s_comment).',
 'customer(1,"Customer#000000001","IVhzIApeRb ot,c,E",15,"25-989-741-2988",711.56,"BUILDING","to the even, regular platelets. regular, ironic epitaphs nag e",black,"-",1,"Customer#000000001","IVhzIApeRb ot,c,E",15,"25-989-741-2988",711.56,"BUILDING","to the even, regular platel